In [110]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [111]:
df = pd.read_csv("co2_ml_data.csv", header=0)
df = df[['CO2TES(tCO2 per TJ)','country','year']]
df.columns


Index(['CO2TES(tCO2 per TJ)', 'country', 'year'], dtype='object')

In [112]:
df

,CO2TES(tCO2 per TJ),country,year
0,59.253,Algeria,1971
1,57.281,Algeria,1972
2,54.459,Algeria,1973
3,54.996,Algeria,1974
4,58.629,Algeria,1975
...,...,...,...
1576,21.237,Zimbabwe,2017
1577,22.844,Zimbabwe,2018
1578,21.277,Zimbabwe,2019
1579,17.693,Zimbabwe,2020


In [113]:
categories = df['country'].values.reshape(-1, 1)

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
one_hot_encoded = encoder.fit_transform(categories)

# Create a new DataFrame with the one-hot encoded columns
columns = [f'country_{category}' for category in encoder.get_feature_names_out(['country'])]
df_encoded = pd.DataFrame(one_hot_encoded, columns=columns)

# Concatenate the one-hot encoded columns with the original DataFrame
df = pd.concat([df, df_encoded], axis=1)

# Drop the original categorical column if needed
df = df.drop('country', axis=1)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [114]:
df

,CO2TES(tCO2 per TJ),year,country_country_Algeria,country_country_Angola,country_country_Benin,country_country_Botswana,country_country_Cameroon,country_country_Democratic Republic of the Congo,country_country_Egypt,country_country_Equatorial Guinea,...,country_country_Rwanda,country_country_Senegal,country_country_South Africa,country_country_South Sudan,country_country_Sudan,country_country_Togo,country_country_Tunisia,country_country_Uganda,country_country_Zambia,country_country_Zimbabwe
0,59.253,1971,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,57.281,1972,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,54.459,1973,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,54.996,1974,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,58.629,1975,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1576,21.237,2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1577,22.844,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1578,21.277,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1579,17.693,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [115]:
# Extracting features and target
y = df['CO2TES(tCO2 per TJ)']
x = df.drop('CO2TES(tCO2 per TJ)', axis=1)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)


In [116]:
y_train.shape

(1264,)

In [117]:
from sklearn.metrics import r2_score
model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
for degree in range(1,5):
  model = make_pipeline(PolynomialFeatures(degree), LinearRegression())

  # Fitting the model
  model.fit(X_train, y_train)

  # Predicting on the test set
  y_pred = model.predict(X_test)

  # Evaluating the model ,Calculate R2 score
  r2 = r2_score(y_test,y_pred)
  # Print the R2 score
  print("R2 Score:", r2," For Degree :",degree)
  # print(f'Mean Squared Error: {mse}')

R2 Score: 0.9192086692438756  For Degree : 1
R2 Score: 0.9599276387041465  For Degree : 2
R2 Score: 0.9693901274260124  For Degree : 3
R2 Score: 0.9692605923245812  For Degree : 4


In [126]:
def predict_function(model,country,year):
    # Example: Predict CO2 emissions for the year 1980 in a specific country
    country_to_predict = country
    year_to_predict = year
    year_to_predict = np.hstack([year])
    # Encode the country using the same LabelEncoder instance
    encoded_country = encoder.transform([[country_to_predict]]).reshape(1, -1)

    # Make the prediction

    for a in encoded_country:
      input_data = np.concatenate([year_to_predict, a]).reshape(1, -1)
      prediction = model.predict(input_data)

      print(f"Predicted CO2 emissions for {country_to_predict} in {year_to_predict[0]}: {prediction[0]}")

In [127]:
predict_function(model,'Algeria',2023)

Predicted CO2 emissions for Algeria in 2023: 53.4493126788293


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(


In [128]:
predict_function(model,'Egypt',2030)

Predicted CO2 emissions for Egypt in 2030: 45.82617642430705


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(


In [129]:
predict_function(model,'Togo',2035)

Predicted CO2 emissions for Togo in 2035: 8.395469980488997


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
